import tensorflow as tf
print(tf.__version__)## COMP90086 Final Project

Benjamin Tam (889835), Matthew Lim (895507)

In [1]:
import os
import numpy as np
import pandas as pd
import cv2  
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers, optimizers
from matplotlib import pyplot as plt

from sklearn import preprocessing 
from sklearn.model_selection import train_test_split

from tqdm.notebook import trange, tqdm

import pixellib
from pixellib.instance import instance_segmentation



ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [3]:
# Get True Label
def get_true_label(label_id):
    return list(train_generator.class_indices.keys())[list(train_generator.class_indices.values()).index(label_id)]

In [4]:
# Initialise image data generator
# With validation split


# Add rotation range

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                               validation_split=0.2)

In [5]:
root = "./"
train_path = './COMP90086_2021_Project_train/train'
test_path = './COMP90086_2021_Project_test/test'

In [6]:
# Import train labels from csv
train_labels = pd.read_csv(os.path.join("./COMP90086_2021_Project_train", "train.csv"), dtype=str)
train_labels['id'] = train_labels['id'].apply(lambda x: '{}.jpg'.format(x))

# Preprocess to get categorical class from coordinates. "x,y" used
train_labels['xy'] = train_labels['x'] + "," + train_labels['y']
train_labels['xy'] = train_labels['xy'].astype('category')

train_labels['cat'] = train_labels['xy'].cat.codes


# Import train labels from csv
test_labels = pd.read_csv(os.path.join("./COMP90086_2021_Project_test", "imagenames.csv"), dtype=str)
test_labels['id'] = test_labels['id'].apply(lambda x: '{}.jpg'.format(x))


# Integer encoding for coordinate label
# train_labels['encoded'] = preprocessing.LabelEncoder().fit_transform(train_labels['xy'])
# train_labels['encoded'] = train_labels['encoded'].apply(str)

In [7]:
# list(train_labels.cat)

In [8]:
batch_size = 20

train_generator = train_datagen.flow_from_dataframe(train_labels, 
                                               directory = train_path,
                                               x_col = 'id', y_col = 'xy',
                                                    target_size=(150,150),
                                                    batch_size=batch_size,
                                                    class_mode="sparse",
                                                   subset='training')
val_generator = train_datagen.flow_from_dataframe(train_labels, 
                                               directory = train_path,
                                               x_col = 'id', y_col = 'xy',
                                                  target_size=(150,150),
                                                  batch_size=batch_size,
                                                  class_mode="sparse",
                                                   subset='validation')

Found 6000 validated image filenames belonging to 1499 classes.
Found 1500 validated image filenames belonging to 1499 classes.


In [9]:
train_imgs, train_lbl = next(iter(train_generator))
val_imgs, val_lbl = next(iter(val_generator))

In [10]:
# To obtain label ID
np.argmax(train_lbl[2])

# To obtain True label
get_true_label(1373)

'66.51932192,-44.01728035'

In [11]:
train_imgs[0]

array([[[0.4901961 , 0.5294118 , 0.5647059 ],
        [0.4901961 , 0.5294118 , 0.5647059 ],
        [0.4901961 , 0.5294118 , 0.5647059 ],
        ...,
        [0.41176474, 0.42352945, 0.39607847],
        [0.41176474, 0.42352945, 0.39607847],
        [0.4039216 , 0.4156863 , 0.38823533]],

       [[0.4901961 , 0.5294118 , 0.5647059 ],
        [0.4901961 , 0.5294118 , 0.5647059 ],
        [0.4901961 , 0.5294118 , 0.5647059 ],
        ...,
        [0.41176474, 0.42352945, 0.39607847],
        [0.41176474, 0.42352945, 0.39607847],
        [0.4039216 , 0.4156863 , 0.38823533]],

       [[0.4901961 , 0.5294118 , 0.5647059 ],
        [0.4901961 , 0.5294118 , 0.5647059 ],
        [0.49411768, 0.53333336, 0.5686275 ],
        ...,
        [0.41176474, 0.42352945, 0.39607847],
        [0.40784317, 0.41960788, 0.3921569 ],
        [0.40784317, 0.41960788, 0.3921569 ]],

       ...,

       [[0.31764707, 0.33333334, 0.3803922 ],
        [0.3254902 , 0.33333334, 0.3803922 ],
        [0.31764707, 0

## Use VGG19 to extract features

In [123]:
import os
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [124]:
# base_model = VGG19(weights='imagenet', input_shape=[150,150,3],include_top=False) 

In [125]:
# extract = Model(base_model.inputs, base_model.get_layer('block5_conv4').output)

In [126]:
# preds = extract.predict(train_generator)

In [141]:
def create_model(input_shape, n_classes, optimizer, fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = VGG16(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
input_shape = (150, 150, 3)
optim_1 = Adam(learning_rate=0.01)
n_classes=1499

n_steps = train_generator.samples // batch_size
n_val_steps = val_generator.samples // batch_size
n_epochs = 50

# First we'll train the model without Fine-tuning
vgg_model = create_model(input_shape, n_classes, optim_1, fine_tune=0)

In [ ]:
# train_generator.samples

In [ ]:
# from livelossplot.inputs.keras import PlotLossesCallback
from livelossplot import PlotLossesKeras
from livelossplot.keras import PlotLossesCallback

plot_loss_1 = PlotLossesCallback()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='tl_model_v1.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

In [ ]:

vgg_history = vgg_model.fit(train_generator,
                            batch_size=batch_size,
                            epochs=n_epochs,
                            validation_data=val_generator,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1, early_stop, plot_loss_1],
                            verbose=1)

In [ ]:
train_generator.reset()
val_generator.reset()

## Instance Segmentation Method

In [12]:
instance_seg = instance_segmentation()
instance_seg.load_model("mask_rcnn_coco.h5")


Instructions for updating:
Use fn_output_signature instead


In [13]:
def get_object_classes(train_path, image_filename, instance_seg, pbar=None):
    segmask, output = instance_seg.segmentImage(os.path.join(train_path, image_filename))
    if pbar:
        pbar.update(1)
    return segmask['class_ids']

In [15]:
pbar = tqdm(total=train_labels.shape[0])
train_labels['objects'] = train_labels.apply(lambda x: get_object_classes(train_path, x['id'], instance_seg, pbar), axis=1)

  0%|          | 0/7500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
train_labels.to_csv("train_labels_objects.csv")

In [ ]:
train_labels.iloc[1235]

In [ ]:
np.unique(train_labels["xy"])

In [ ]:
d = {}

d['hello'] = 2

In [ ]:
train_labels

In [ ]:

# coord_objects = {}
# for item in train_labels.values:
#     try:
#         coord_objects[item[3]] = np.concatenate((coord_objects[item[3]], item[4]))
#     except KeyError:
#         coord_objects[item[3]] = item[4]

In [ ]:
a=coord_objects['-9.380678081,3.58271965']

In [ ]:
b=[1, 1, 1, 1, 78]

In [ ]:
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(a, b)/(norm(a)*norm(b))

In [ ]:
segmask, output = instance_seg.segmentImage(os.path.join(train_path, "IMG2744_2.jpg"))

In [ ]:
segmask['class_ids']

## Method 2: SIFT keypoints and KNN

In [17]:
sift = cv2.SIFT_create()

In [18]:
def sift_compute(image):
    
    return sift.detectAndCompute(image, None)
    

In [19]:
def sift_read_image(path, file_name, pbar=None):
    image = cv2.imread(os.path.join(path, file_name), cv2.IMREAD_GRAYSCALE)
    if pbar:
        pbar.update(1)
    return (sift.detectAndCompute(image, None))

In [20]:
# Find key points and descriptors for all items in training set
# kp desc
pbar = tqdm(total=7500)
train_labels['sift_data']=[sift_read_image(train_path, file_name, pbar) for file_name in train_labels['id']]
pbar.close()

  0%|          | 0/7500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
train_labels['sift_data'][0][1]

In [ ]:
# Split DF into train val set
train_df, val_df = train_test_split(train_labels, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
def flann_match_only(des_query, des_train, pbar=None):
    try:
        des1 = des_query
        des2 = des_train

        # FLANN parameters and initialize
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)

        # Matching descriptor using KNN algorithm
        matches = flann.knnMatch(des1,des2,k=2)   
        return len(matches)
    except:
        return 0
    finally:
        if pbar:
            pbar.update(1)

In [ ]:
# Use FLANN based matcher 
def flann_matches(des_query, des_train, pbar=None):  
    try:
        des1 = des_query
        des2 = des_train

        # FLANN parameters and initialize
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = {} #dict(checks=40)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)

        # Matching descriptor using KNN algorithm
        matches = flann.knnMatch(des1,des2,k=2)

        # Get good amtches
        # Store all good matches as per Lowe's Ratio test.
        good=0
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good+=1
                
        return good
    except:
        # If no features
        return 0 
    finally:   
        if pbar:
            pbar.update(1)

In [ ]:
def get_predicted_loc(path, query_image_filename, train_labels):
    
    
    # Find matches between query image and all training images
    kp_query, des_query = sift_read_image(path, query_image_filename)
    
    pbar = tqdm(total=len(train_labels.index))
    matches=[((item[0], item[1]),flann_matches(des_query, item[2][1], pbar)) for item in train_labels[['x', 'y', 'sift_data']].to_numpy()]
#     matches=[((item[0], item[1]),flann_match_only(des_query, item[2][1], pbar)) for item in train_labels[['x', 'y', 'sift_data']].to_numpy()]
    pbar.close()
    
    
    # Get match list and store it in a dict to add up all keypoints for each location
    return sorted(matches, key=lambda x:x[1], reverse=True)[0]
    
#     from collections import defaultdict
#     result_dict = defaultdict(int)

#     for i in matches:
#         result_dict[i[0]] += i[1] 

#     # Find x y coord with most matched keypoints as per flann matcher
# #     out = max(result_dict, key=result_dict.get)
    
#     out = sorted(result_dict.items(), key=lambda item: item[1], reverse=True)
    
    return out

In [ ]:
kp_query, des_query = sift_read_image(test_path, "IMG4399_1.jpg")
kp_query2, des_query2 = sift_read_image(test_path, "IMG4287_3.jpg")

In [ ]:
flann_match_only(des_query, des_query2)

In [ ]:
result = get_predicted_loc(test_path, "IMG4569_3.jpg", train_labels)

In [ ]:
print(result)

In [ ]:
# # Get match list and store it in a dict to add up all keypoints for each location
# sorted(match_list, key=lambda x:x[1], reverse=True)
# from collections import defaultdict
# result_dict = defaultdict(int)

# for i in match_list:
#     result_dict[i[0]] += i[1] 

# # Find x y coord with most matched keypoints as per flann matcher
# max(result_dict, key=result_dict.get)

In [ ]:
test_results=[]
for i in 

## Benchmark Method

In [ ]:
# import the necessary packages
import imageio
import glob
import os
import time
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from tensorflow import keras

In [ ]:
def encoder(input_encoder):
    
    inputs = keras.Input(shape=input_encoder, name='input_layer')
    # Block 1
    x = layers.Conv2D(32, kernel_size=3, strides= 1, padding='same', name='conv_1')(inputs)
    x = layers.BatchNormalization(name='bn_1')(x)
    x = layers.LeakyReLU(name='lrelu_1')(x)
   
    # Block 2
    x = layers.Conv2D(64, kernel_size=3, strides= 2, padding='same', name='conv_2')(x)
    x = layers.BatchNormalization(name='bn_2')(x)
    x = layers.LeakyReLU(name='lrelu_2')(x)
   
    # Block 3
    x = layers.Conv2D(64, 3, 2, padding='same', name='conv_3')(x)
    x = layers.BatchNormalization(name='bn_3')(x)
    x = layers.LeakyReLU(name='lrelu_3')(x)
    
    # Block 4
    x = layers.Conv2D(64, 3, 1, padding='same', name='conv_4')(x)
    x = layers.BatchNormalization(name='bn_4')(x)
    x = layers.LeakyReLU(name='lrelu_4')(x)
    
    # Final Block
    flatten = layers.Flatten()(x)
    bottleneck = layers.Dense(2, name='dense_1')(flatten)
    model = tf.keras.Model(inputs, bottleneck, name="Encoder")
    return model

In [ ]:
def decoder(input_decoder):
    # Initial Block
    inputs = keras.Input(shape=input_decoder, name='input_layer')
    x = layers.Dense(3136, name='dense_1')(inputs)
    x = tf.reshape(x, [-1, 7, 7, 64], name='Reshape_Layer')
    
    # Block 1
    x = layers.Conv2DTranspose(64, 3, strides= 1, padding='same',name='conv_transpose_1')(x)
    x = layers.BatchNormalization(name='bn_1')(x)
    x = layers.LeakyReLU(name='lrelu_1')(x)
    
    # Block 2
    x = layers.Conv2DTranspose(64, 3, strides= 2, padding='same', name='conv_transpose_2')(x)
    x = layers.BatchNormalization(name='bn_2')(x)
    x = layers.LeakyReLU(name='lrelu_2')(x)
   
    # Block 3
    x = layers.Conv2DTranspose(32, 3, 2, padding='same', name='conv_transpose_3')(x)
    x = layers.BatchNormalization(name='bn_3')(x)
    x = layers.LeakyReLU(name='lrelu_3')(x)
   
    # Block 4
    outputs = layers.Conv2DTranspose(1, 3, 1,padding='same', activation='sigmoid', name='conv_transpose_4')(x)
    model = tf.keras.Model(inputs, outputs, name="Decoder")
    return model

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005)
def ae_loss(y_true, y_pred):
    loss = K.mean(K.square(y_true - y_pred), axis = [1,2,3])
    return loss

In [ ]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):

    with tf.GradientTape() as encoder, tf.GradientTape() as decoder:
      
        latent = enc(images, training=True)
        generated_images = dec(latent, training=True)
        loss = ae_loss(images, generated_images)
        
    gradients_of_enc = encoder.gradient(loss, enc.trainable_variables)
    gradients_of_dec = decoder.gradient(loss, dec.trainable_variables)
    
    
    optimizer.apply_gradients(zip(gradients_of_enc, enc.trainable_variables))
    optimizer.apply_gradients(zip(gradients_of_dec, dec.trainable_variables))
    return loss

In [ ]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()
    for image_batch in dataset:
      train_step(image_batch)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    
epoch = 20
train(train_generator, epoch)

In [ ]:
cnn = keras.Sequential(
    [
        layers.InputLayer((150, 150, 3)),
        
        layers.Conv2D(8, (5, 5), activation='relu'), # fill in
        layers.MaxPooling2D((2, 2)), # fill in
        layers.Conv2D(16, (5, 5), activation='relu'), #fill in
        layers.MaxPooling2D((2, 2)), # fill in
        
        layers.Flatten(),
        layers.Dense(1499, activation='softmax')
    ], 
)

In [ ]:
cnn.compile(optimizer='adam',
           loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), #use SparseCategoricalCrossentropy because labels are integers. If the labels are one-hot representation, please use CategoricalCrossentropy loss.
           metrics=['accuracy'])

In [ ]:
history= model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
#         steps_per_epoch=10,
        epochs=20,
        validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
        batch_size=batch_size)
#         validation_steps=10)